# QRCodeStaticCode Maker

@alessioborgi

QRStaticCode allows you to creating static, unlimited-use QR codes with custom styles, embedded icons, and optional captions.


### 1: IMPORTING LIBRARIES

In [ ]:
from qrcode import QRCode
from qrcode.image.styledpil import StyledPilImage
from qrcode.image.styles.moduledrawers import RoundedModuleDrawer
from qrcode.image.styles.colormasks import SolidFillColorMask
from PIL import Image, ImageDraw, ImageFont

### 2: FUNCTIONS DEFINITION

In [ ]:
def generate_stylish_qr(
    url: str,                                   # URL to encode.
    output_path: str = "qr_with_text.png",      # Output file path.
    box_size: int = 20,                         # Size of each QR module in pixels.
    border: int = 4,                            # Size of the border (minimum is 4).
    fill_color: str = "#1f2937",              # QR module color.
    back_color: str = "white",                  # Background color.
    icon_path: str = None,                      # Path to icon image (optional).
    icon_color: str = None,                     # Tint color for icon (defaults to fill_color).
    icon_size_ratio: float = 0.25,              # Icon size as a fraction of QR size.
    icon_border: int = 10,                      # Space between icon and QR modules.
    caption: str = None,                        # Caption text (optional).
    font_path: str = None,                      # Path to TTF font file (optional).
    font_size: int = 24,                        # Font size for caption.
    caption_color: str = "#1f2937",           # Caption text color.
    caption_padding: int = 10                   # Space between QR and text.
):
    # 1) Build the QR code.
    qr = QRCode(
        error_correction=QRCode.ERROR_CORRECT_H,
        box_size=box_size,
        border=border,
    )
    qr.add_data(url)
    qr.make(fit=True)

    # 2) Render it with rounded modules.
    qr_img = qr.make_image(
        image_factory=StyledPilImage,
        module_drawer=RoundedModuleDrawer(),
        color_mask=SolidFillColorMask(front_color=fill_color, back_color=back_color)
    ).convert("RGBA")

    # 3) (Optional) Overlay icon.
    if icon_path:
        w, h = qr_img.size
        icon = Image.open(icon_path).convert("RGBA")

        # Recolor the icon to icon_color (or fill_color by default).
        tint = icon_color or fill_color
        # Extract alpha channel as mask.
        _, _, _, alpha = icon.split()
        # Create a solid‐color image, then apply the original icon's alpha.
        colored_icon = Image.new("RGBA", icon.size, tint)
        colored_icon.putalpha(alpha)
        icon = colored_icon

        # Resize to fit.
        icon_w = int(w * icon_size_ratio)
        icon = icon.resize((icon_w, icon_w), Image.ANTIALIAS)

        # Clear a circle behind it.
        draw = ImageDraw.Draw(qr_img)
        cx, cy = w // 2, h // 2
        r = icon_w // 2 + icon_border
        draw.ellipse((cx-r, cy-r, cx+r, cy+r), fill=back_color)

        # Paste tinted icon.
        qr_img.paste(icon, (cx - icon_w//2, cy - icon_w//2), icon)

    # 4) If no caption, just save.
    if not caption:
        qr_img.save(output_path)
        print(f"Saved QR to {output_path}")
        return

    # 5) Load font (fallback to default).
    try:
        font = ImageFont.truetype(font_path or "", font_size)
    except Exception:
        font = ImageFont.load_default()

    # 6) Measure text.
    draw_dummy = ImageDraw.Draw(qr_img)
    text_w, text_h = draw_dummy.textsize(caption, font=font)

    # 7) Create new canvas taller by text height + padding.
    qr_w, qr_h = qr_img.size
    new_h = qr_h + caption_padding + text_h
    canvas = Image.new("RGBA", (qr_w, new_h), back_color)

    # 8) Paste QR at top.
    canvas.paste(qr_img, (0, 0))

    # 9) Draw caption centered.
    draw = ImageDraw.Draw(canvas)
    x = (qr_w - text_w) // 2
    y = qr_h + caption_padding
    draw.text((x, y), caption, font=font, fill=caption_color)

    # 10) Save.
    canvas.save(output_path)
    print(f"Saved QR with caption to {output_path}")


### 3: MAIN (EXAMPLE USAGE)

In [ ]:
url = "https://alessioborgi.github.io/Z-SASLM.github.io/"          # URL to encode.                    
output_path: str = "Z-SASLM_QR_Code.png"                           # Output file path.
fill_color: str = "#1f2937",                                     # QR module color.   
icon_path: str = None,                      # Path to icon image (optional).
icon_size_ratio: float = 0.25,              # Icon size as a fraction of QR size.
icon_border: int = 10,                      # Space between icon and QR modules.
caption: str = None,                        # Caption text (optional).
font_path: str = None,                      # Path to TTF font file (optional).
font_size: int = 24,                        # Font size for caption.    
caption_color: str = "#1f2937",           # Caption text color. 
caption_padding: int = 10                   # Space between QR and text.


# Other optional parameters.
box_size = 20                                                # Size of each QR module in pixels. 
border = 4                            # Size of the border (minimum is 4).  
back_color: str = "white",                                      # Background color.    
icon_color = None,    # Tint color for icon (defaults to fill_color).

In [ ]:

# Example usage:
if __name__ == "__main__":
    generate_stylish_qr(
        url="https://example.com",
        icon_path="globe.png",
        caption="Scan Me!",
        font_path="/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",
        font_size=32
    )
